# Features Engineering

Dans cette section, nous allons effectuer l'ingénierie des caractéristiques sur notre dataset. Cette étape est cruciale pour transformer et créer de nouvelles variables qui pourront améliorer les performances de nos modèles. Nous allons créer des features pertinentes, normaliser les données si nécessaire, et préparer notre dataset pour la phase de modélisation.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

data = pd.read_csv('../RAW_data/exepeditionsV2.csv', sep=";")
data.head()